In [2]:
import json
import gzip
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


In [3]:
rows = []
import ast
with open("Datasets/australian_user_reviews.json", encoding='MacRoman') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
df_user_reviews = df_reviews.explode("reviews")
df_user_reviews = pd.concat([df_user_reviews.drop(["reviews"], axis=1), df_user_reviews["reviews"].apply(pd.Series)], axis=1)
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [5]:
df_user_reviews.columns

Index([    'user_id',    'user_url',       'funny',      'posted',
       'last_edited',     'item_id',     'helpful',   'recommend',
            'review',             0],
      dtype='object')

In [6]:
df_user_reviews = df_user_reviews.iloc[:, :-1]

## Trabajo con la columna "item_id"

In [7]:
df_user_reviews.dropna(subset="item_id", inplace=True)

In [8]:
len(df_user_reviews["item_id"])

59305

## Realizo el "sentiment_analysis" , con la columna "review"

In [9]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jeffe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
sia = SentimentIntensityAnalyzer()

In [12]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [13]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['sentiment_analysis'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

In [14]:
df_user_reviews['sentiment_analysis'].unique()

array([2, 1, 0], dtype=int64)

## Creo mi dataframe limpio

In [15]:
df_user_reviews_final = df_user_reviews.drop(["user_url", "funny", "posted",	"last_edited",	"helpful",	"review", ], axis=1)

In [16]:
df_user_reviews_final.to_csv('df_user_reviews_clean.csv', sep=',' , index=False)

In [17]:
df_user_reviews_clean = pd.read_csv('df_user_reviews_clean.csv')

In [18]:
df_user_reviews_clean

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
59300,76561198312638244,70,True,2
59301,76561198312638244,362890,True,2
59302,LydiaMorley,273110,True,2
59303,LydiaMorley,730,True,2


## Trabajo con la columna "posted"

In [19]:
#df_reviews["posted"] = df_reviews["posted"].astype(str)

In [20]:
#['posted'].map(lambda x: str.split(x))

In [21]:
#df_reviews['posted'] = df_reviews['posted'].str.rstrip('.')

In [22]:
#df_reviews["year"] = pd.to_datetime(df_reviews["posted"], format="Posted %B %d, %Y", errors="coerce").dt.year

In [23]:
#df_reviews["posted"] = df_reviews["year"].apply(lambda x: x)

In [24]:
#print(df_reviews.iloc[0]['review'])